<a href="https://colab.research.google.com/github/ngatclc3/echocardiogram/blob/main/echocardiogram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import library and *package*

In [2]:
import torch
import matplotlib.pyplot as plt

from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder

In [26]:
from google.colab import drive
drive.mount('/content/drive')
train_path = "/content/drive/MyDrive/Colab/assets/DATA_CHAMBER_2021/train"
test_path = "/content/drive/MyDrive/Colab/assets/DATA_CHAMBER_2021/test"

Mounted at /content/drive


In [30]:
from torchvision import transforms

# Generate different transformations for each data type
CJ = transforms.ColorJitter()
Rs = transforms.Resize((224, 224))
CC = transforms.CenterCrop(size=224)
RR = transforms.RandomRotation(degrees=15)
RRC = transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0))

RC = transforms.RandomCrop(32, padding=4)
RHF = transforms.RandomHorizontalFlip()
RVF = transforms.RandomVerticalFlip()
TPI = transforms.ToPILImage()
TT = transforms.ToTensor()
Nml = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))

transform_train_preprocess =  transforms.Compose([Rs, CJ, RR, TT, Nml])
transform_train_rescale =  transforms.Compose([Rs, RRC, CC, TT, Nml])
transform_train_with_aug = transforms.Compose([Rs, TPI, RC, RHF, TT, Nml])
transform_train_no_aug = transforms.Compose([Rs, TT, Nml])

transform_test = transforms.Compose([Rs, TT, Nml])

In [23]:
from torchvision.datasets import ImageFolder
from collections import namedtuple

TrainTest = namedtuple('TrainTest', ['train', 'test'])
classes = ['2C','3C','4C']
n_classes = len(classes)


def myloader(filename):
    return Image.open(filename).convert('RGB')

def prepare_data(transform_train):
  trainset = ImageFolder(root = train_path, transform = transform_train, loader=myloader)
  testset = ImageFolder(root = test_path, transform = transform_test, loader=myloader)
  return TrainTest(train = trainset, 
                   test = testset)
  
def prepare_loader(datasets):
  batch = 32
  worker = 2
  trainloader = DataLoader(dataset = datasets.train,
                           batch_size = batch, 
                           shuffle = True, 
                           num_workers = worker)
  testloader = DataLoader(dataset = datasets.test, 
                          batch_size = batch, 
                          shuffle = False, 
                          num_workers = worker)
  return TrainTest(train = trainloader, 
                   test = testloader)
  


In [5]:
from torchvision import models

# Use pretrained models which are available in torchvision
pretrained_models = {
    'alexnet': models.alexnet(pretrained=True),
    # 'vgg16': models.vgg16(pretrained=True),
    # 'googlenet': models.googlenet(pretrained=True),
    # 'resnet18': models.resnet18(pretrained=True),
    # 'densenet': models.densenet161(pretrained=True),
    # 'inception': models.inception_v3(pretrained=True),

}

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

In [ ]:
pretrained_models.get('alexnet').eval()

In [21]:
class Alexnet(nn.Module):
    def __init__(self):
        super(Alexnet, self).__init__()
        self.model = models.alexnet()
        self.fc1 = nn.Linear(in_features=1000, out_features=n_classes)
        
    def forward(self, x):
        out = self.model
        out = self.fc1(out)
        return out

In [ ]:
def get_model(model):

In [8]:
from torch.nn import CrossEntropyLoss
from torch.optim import SGD

def get_trainer(model):
    loss = CrossEntropyLoss()
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
    return loss, optimizer



In [28]:
def train_epoch(epoch, model, loader, device):
    model.train()
    running_loss = 0.0
    running_acc = 0

    loss, optimizer = get_trainer(model)
    
    for i, (images, labels) in enumerate(loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss_train = loss(outputs, labels)

        running_loss += loss_train.item()
        _, preds = torch.max(outputs, 1)
        running_acc += (preds == labels).sum().item()
        loss_train.backward()
        optimizer.step()

    train_loss = running_loss/len(loader.dataset)
    train_accuracy = 100. * running_acc/len(loader.dataset)

    print(f'Training Loss: {train_loss:.4f}, Training Acc: {train_accuracy:.2f}')
    
    return train_loss, train_accuracy

In [18]:
def test_epoch(epoch, model, loader, device):
    model.eval()
    running_loss = 0.0
    running_acc = 0

    loss, _ = get_trainer(model)

    for i, (images, labels) in enumerate(loader):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)

        loss_test = loss(outputs, labels)
        
        running_loss += loss_test.item()
        _, predicted = torch.max(outputs, dim=1)
        running_acc += (predicted == labels).sum().item()

    test_loss = running_loss/len(loader.dataset)
    test_accuracy = 100. * running_acc/len(loader.dataset)

    return test_loss, test_accuracy

In [ ]:
import time

def run(model):
    datasets = prepare_data(transform_train_no_aug)
    loaders = prepare_loader(datasets)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device=device)

    training_loss, training_accuracy = [], []
    testing_loss, testing_accuracy = [], []
    
    start = time.time()

    for epoch in range(10):
        train_loss, train_acc = train_epoch(epoch, model, loaders.train, device)
        test_loss, test_acc = test_epoch(epoch, model, loaders.test, device)

        training_loss.append(train_loss)
        training_accuracy.append(train_acc)
        testing_loss.append(test_loss)
        testing_accuracy.append(test_acc)
    
    end = time.time()
    print((end-start)/60, 'minutes')

run(Alexnet())